In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import datasets
from keras.layers import Input, Dense
from keras.models import Model

/Users/hbi16859/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = datasets.load_digits()

In [3]:
X_data = data.images
y_data = data.target

X_data = X_data.reshape(X_data.shape[0], 64)

In [4]:
# fit in data instances into interval [0,1]
X_data = X_data / 16.
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3, random_state = 777)

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1257, 64)
(540, 64)
(1257,)
(540,)


In [8]:
# define coding dimension. Coding dimension will be the size of reduced data dimension
code_dim = 16

In [9]:
#linear model
def auto_encoder_model():    
    inputs = Input(shape = (X_train.shape[1],), name = 'input')                         # input layer
    code = Dense(code_dim, activation = 'relu', name = 'code')(inputs)  # hidden layer => represents "codes"
    outputs = Dense(X_train.shape[1], activation = 'softmax', name = 'output')(code)    # output layer

    auto_encoder = Model(inputs = inputs, outputs = outputs)

    encoder = Model(inputs = inputs, outputs = code)

    decoder_input = Input(shape = (code_dim,))
    decoder_output = auto_encoder.layers[-1]
    decoder = Model(inputs = decoder_input, outputs = decoder_output(decoder_input))

    auto_encoder.compile(optimizer='adam', loss='binary_crossentropy')
    return encoder, decoder, auto_encoder

In [10]:
encoder, decoder, auto_encoder = auto_encoder_model()

In [12]:
auto_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 64)                0         
_________________________________________________________________
code (Dense)                 (None, 16)                1040      
_________________________________________________________________
output (Dense)               (None, 64)                1088      
Total params: 2,128
Trainable params: 2,128
Non-trainable params: 0
_________________________________________________________________


In [13]:
%%time
history=auto_encoder.fit(X_train, X_train, epochs = 20, batch_size = 50, validation_data = (X_test, X_test), verbose = 0)

CPU times: user 1.5 s, sys: 669 ms, total: 2.17 s
Wall time: 1.19 s


In [14]:
# generate reduced data by using "encoders"
training_data_reduced = encoder.predict(X_train)
test_data_reduced = encoder.predict(X_test)